In [1]:
# !pip install snowflake-connector-python

In [2]:
import snowflake.connector
import pandas as pd
import json

'''user = REFRACT.FOSFOR@LNTINFOTECH.COM
password = Password321
account = fya62509.us-east-1
warehouse = FOSFOR_REFRACT
database = FOSFOR_REFRACT
schema = SALES
role = FOSFOR_REFRACT'''    

# Snowflake connection parameters
snowflake_account = 'rwb76633'
snowflake_user = 'Ravi'
snowflake_password = 'Kayakave@1983'
snowflake_database = 'TUTORIAL_DB'
snowflake_schema = 'DATA_SCHEMA'
snowflake_stage = 'MAGICODER2'
role = 'TEST_ROLE'

# snowflake_account = 'fya62509.us-east-1'
# snowflake_user = 'REFRACT.FOSFOR@LNTINFOTECH.COM'
# snowflake_password = 'Password321'
# snowflake_database = 'FOSFOR_REFRACT'
# snowflake_schema = 'SALES'
# snowflake_stage = 'MAGICODER'
 
# Connect to Snowflake
conn = snowflake.connector.connect(
   user=snowflake_user,
   password=snowflake_password,
   account=snowflake_account,
   session_parameters={
      'PYTHON_CONNECTOR_QUERY_RESULT_FORMAT': 'json'
   },
   database=snowflake_database,
   schema=snowflake_schema,
   role=role,
)




In [3]:
conn

In [3]:
#! ls -lrt /llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66

In [4]:
# Create a cursor object
cur = conn.cursor()
 
# Upload file to stage (example using Pandas)
#file_path = '/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66/special_tokens_map.json'
# df = pd.read_csv(file_path)
#cur.execute(f"PUT file://{file_path} @{snowflake_stage}/magicoder_model AUTO_COMPRESS = FALSE")
#cur.execute("Select * from Sales")

In [170]:
prompt=""""Assume you are a DBT query optimizer, Optimize this DBT query and if possible modularize this query using CTE: '''
SELECT
ACC.ACCOUNT_ID AS ACCOUNT_ID
,CASE WHEN OUTER.ACCOUNT_ID IS NOT NULL THEN 'Y' ELSE 'N' END AS DONATED
,NVL(NULLIF(SPLIT_PART(OUTER.FIRST_OPPORTUNITY_DTL, '|', 1), 0),0) AS FIRST_GIFT_AMOUNT
,NULLIF(SPLIT_PART(OUTER.FIRST_OPPORTUNITY_DTL, '|', 2),'') AS FIRST_GIFT_DATE
,NVL(OUTER.TOTAL_AMOUNTS_OF_GIFTS_ALL_TIME_HC,0) AS TOTAL_AMOUNTS_OF_GIFTS_ALL_TIME_HC
,NVL(OUTER.TOTAL_NUMBER_OF_GIFTS_ALL_TIME_HC,0) AS TOTAL_NUMBER_OF_GIFTS_ALL_TIME_HC
,NVL(OUTER.TOTAL_AMOUNTS_OF_GIFTS_ALL_TIME_SC,0) AS TOTAL_AMOUNTS_OF_GIFTS_ALL_TIME_SC
,NVL(OUTER.TOTAL_NUMBER_OF_GIFTS_ALL_TIME_SC,0) AS TOTAL_NUMBER_OF_GIFTS_ALL_TIME_SC
FROM ""DIM_DB"".""CORE"".""DIM_ACCOUNT"" ACC
LEFT JOIN (SELECT
INNER.ACCOUNT_ID
,ANY_VALUE(INNER.FIRST_OPPORTUNITY_DTL) AS FIRST_OPPORTUNITY_DTL
,ANY_VALUE(INNER.LAST_OPPORTUNITY_DTL) AS LAST_OPPORTUNITY_DTL
,ANY_VALUE(INNER.SMALLEST_OPPORTUNITY_DTL) AS SMALLEST_OPPORTUNITY_DTL
,ANY_VALUE(INNER.LARGEST_OPPORTUNITY_DTL)  AS LARGEST_OPPORTUNITY_DTL
,AVG(INNER.OPPORTUNITY_AMOUNT) AS AVERAGE_OPPORTUNITY_AMOUNT
,ARRAY_SIZE(ARRAY_AGG( DISTINCT INNER.FISCAL_YEAR)  WITHIN GROUP (ORDER BY INNER.FISCAL_YEAR DESC)) AS NUMBER_OF_YEARS_DONATING
,SUM(INNER.CURR_PREV_CONSECUTIVE_YEAR_DONATING_FLAG) AS CURR_PREV_NUMBER_OF_CONSECUTIVE_YEAR_DONATING
,""DIM_DB"".""CORE"".""NUM_OF_MAX_CONSQ_YEARS_CALC_FN""(ARRAY_AGG( DISTINCT INNER.FISCAL_YEAR) WITHIN GROUP (ORDER BY INNER.FISCAL_YEAR DESC)) AS MAXIMUM_NUMBER_OF_CONSECUTIVE_YEAR_DONATING
,GET(ARRAY_AGG(INNER.CAL_YEAR) WITHIN GROUP (ORDER BY INNER.TOTAL_DONATION_OVER_CAL_YEAR DESC, INNER.CAL_YEAR DESC), 0) AS BEST_GIFT_CAL_YEAR
,MAX(INNER.TOTAL_DONATION_OVER_CAL_YEAR) AS BEST_GIFT_CAL_YEAR_TOTAL_AMOUNT
,GET(ARRAY_AGG(INNER.FISCAL_YEAR) WITHIN GROUP (ORDER BY INNER.TOTAL_DONATION_OVER_FISCAL_YEAR DESC, INNER.FISCAL_YEAR DESC),0) AS BEST_GIFT_FISCAL_YEAR
,MAX(INNER.TOTAL_DONATION_OVER_FISCAL_YEAR) AS BEST_GIFT_FISCAL_YEAR_TOTAL_AMOUNT
,SUM(INNER.AMOUNTS_OF_GIFTS_ALL_TIME_HC) AS TOTAL_AMOUNTS_OF_GIFTS_ALL_TIME_HC
,SUM(INNER.IS_DONATION_OF_GIFTS_ALL_TIME_HC) AS TOTAL_NUMBER_OF_GIFTS_ALL_TIME_HC
,SUM(INNER.AMOUNTS_OF_GIFTS_ALL_TIME_SC) AS TOTAL_AMOUNTS_OF_GIFTS_ALL_TIME_SC
,SUM(INNER.IS_DONATION_OF_GIFTS_ALL_TIME_SC) AS TOTAL_NUMBER_OF_GIFTS_ALL_TIME_SC
,ANY_VALUE(INNER.CURR_CAL_YEAR) AS CURR_CAL_YEAR
,ANY_VALUE(INNER.CURR_FISCAL_YEAR) AS CURR_FISCAL_YEAR
FROM
(SELECT
REV.ACCOUNT_ID
,FIRST_VALUE(REV.CONC_DTL) OVER(PARTITION BY REV.ACCOUNT_ID ORDER BY REV.OPPORTUNITY_CLOSEDATE, REV.OPPORTUNITY_ID ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS FIRST_OPPORTUNITY_DTL
,LAST_VALUE(REV.CONC_DTL) OVER(PARTITION BY REV.ACCOUNT_ID ORDER BY REV.OPPORTUNITY_CLOSEDATE, REV.OPPORTUNITY_ID ROWS BETWEEN CURRENT ROW AND UNBOUNDED FOLLOWING) AS LAST_OPPORTUNITY_DTL
,FIRST_VALUE(REV.CONC_DTL) OVER(PARTITION BY REV.ACCOUNT_ID ORDER BY REV.OPPORTUNITY_AMOUNT, REV.OPPORTUNITY_CLOSEDATE, REV.OPPORTUNITY_ID ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS SMALLEST_OPPORTUNITY_DTL
,LAST_VALUE(REV.CONC_DTL) OVER(PARTITION BY REV.ACCOUNT_ID ORDER BY REV.OPPORTUNITY_AMOUNT, REV.OPPORTUNITY_CLOSEDATE, REV.OPPORTUNITY_ID ROWS BETWEEN CURRENT ROW AND UNBOUNDED FOLLOWING) AS LARGEST_OPPORTUNITY_DTL
,CASE WHEN FIRST_VALUE(REV.FISCAL_YEAR) OVER (PARTITION BY REV.ACCOUNT_ID ORDER BY REV.FISCAL_YEAR DESC) < (YEAR(CURRENT_DATE()) -1) THEN 0 WHEN  REV.FI
SCAL_YEAR + DENSE_RANK() OVER (PARTITION BY REV.ACCOUNT_ID ORDER BY REV.FISCAL_YEAR DESC) - ROW_NUMBER() OVER (PARTITION BY REV.ACCOUNT_ID,REV.FISCAL_YE
AR ORDER BY REV.FISCAL_YEAR DESC) = FIRST_VALUE(REV.FISCAL_YEAR) OVER (PARTITION BY REV.ACCOUNT_ID ORDER BY REV.FISCAL_YEAR DESC) THEN 1 ELSE 0 END CURR_PREV_CONSECUTIVE_YEAR_DONATING_FLAG
,SUM(REV.OPPORTUNITY_AMOUNT) OVER(PARTITION BY REV.ACCOUNT_ID, REV.CAL_YEAR) AS TOTAL_DONATION_OVER_CAL_YEAR
,SUM(REV.OPPORTUNITY_AMOUNT) OVER(PARTITION BY REV.ACCOUNT_ID, REV.FISCAL_YEAR) AS TOTAL_DONATION_OVER_FISCAL_YEAR
,CASE WHEN REV.OPPORTUNITY_TYPE_HARD_SOFT = 'Direct Credit' THEN REV.OPPORTUNITY_AMOUNT ELSE 0 END AS AMOUNTS_OF_GIFTS_ALL_TIME_HC
,CASE WHEN REV.OPPORTUNITY_TYPE_HARD_SOFT = 'Direct Credit' THEN 1 ELSE 0 END AS IS_DONATION_OF_GIFTS_ALL_TIME_HC
,CASE WHEN REV.OPPORTUNITY_TYPE_HARD_SOFT = 'Indirect Credit' THEN REV.OPPORTUNITY_AMOUNT ELSE 0 END AS AMOUNTS_OF_GIFTS_ALL_TIME_SC
,CASE WHEN REV.OPPORTUNITY_TYPE_HARD_SOFT = 'Indirect Credit' THEN 1 ELSE 0 END AS IS_DONATION_OF_GIFTS_ALL_TIME_SC
,REV.OPPORTUNITY_AMOUNT
,REV.CAL_YEAR
,REV.FISCAL_YEAR
,REV.CURR_CAL_YEAR
,REV.CURR_FISCAL_YEAR
FROM
(SELECT
ARV.OPPORTUNITY_TYPE_HARD_SOFT
,ARV.OPPORTUNITY_DONATIONTYPE
,ARV.ACCOUNT_ID
,ARV.OPPORTUNITY_AMOUNT
,ARV.OPPORTUNITY_CLOSEDATE
,ARV.OPPORTUNITY_CAMPAIGN_NAME
,ARV.OPPORTUNITY_CAMPAIGN_DIVISIONAL_PROGRAM
,ARV.OPPORTUNITY_ID
,ARV.OPPORTUNITY_VEHICLE_TYPE
,ARV.OPPORTUNITY_VEHICLE_SUBTYPE
,C.CONTENT_MESSAGING_CATEGORY__C AS CONTENT_MESSAGING_CATEGORY
,YEAR(ARV.OPPORTUNITY_CLOSEDATE) AS CAL_YEAR
,DD.FISCAL_YEAR AS FISCAL_YEAR
,YEAR(CURRENT_DATE()) AS CURR_CAL_YEAR
,DD_CURR.FISCAL_YEAR AS CURR_FI.SCAL_YEAR
,TO_CHAR(CURRENT_DATE(),'MM-DD') AS CURR_DATE_MM_DD
,NVL(ARV.OPPORTUNITY_AMOUNT,0) || '|' || NVL(ARV.OPPORTUNITY_CLOSEDATE ::VARCHAR, '') || '|' || NVL(ARV.OPPORTUNITY_CAMPAIGN_ID, '') || '|' || NVL(ARV.O
PPORTUNITY_CAMPAIGN_NAME, '') || '|' || NVL(C.CONTENT_MESSAGING_CATEGORY__C, '') || '|' || NVL(ARV.OPPORTUNITY_CAMPAIGN_DIVISIONAL_PROGRAM, '') || '|' |
|  NVL(ARV.OPPORTUNITY_ID, '') || '|' || NVL(ARV.OPPORTUNITY_VEHICLE_TYPE, '') || '|' || NVL(ARV.OPPORTUNITY_VEHICLE_SUBTYPE, '') || '|' || NVL(DD.FISCAL_YEAR::VARCHAR,'')  AS CONC_DTL
FROM REPORTING_DB.SEMANTIC.ALL_REVENUE_V ARV
LEFT JOIN DIM_DB.CORE.DIM_DATE DD ON ARV.OPPORTUNITY_CLOSEDATE = DD.DATE
LEFT JOIN DIM_DB.CORE.DIM_DATE DD_CURR ON CURRENT_DATE() = DD_CURR.DATE
LEFT JOIN STG_DB.CRM_STAGE.CAMPAIGN_S C ON ARV.OPPORTUNITY_CAMPAIGN_ID = C.ID
WHERE ARV.ACCOUNT_ID IS NOT NULL
AND (NVL(ARV.OPPORTUNITY_AMOUNT,0) <> 0)
AND NVL(ARV.OPPORTUNITY_CAMPAIGN_DIVISIONAL_PROGRAM, 'DUMMY') NOT IN ('Type_Kind','Type_Gifts')
AND NVL(ARV.OPPORTUNITY_DONATIONTYPE, 'DUMMY') <> 'Type_Gifts'
AND NVL(ARV.OPPORTUNITY_VEHICLE_TYPE, 'DUMMY') NOT IN ('Type_Kind','Type_Gifts')) AS REV
) AS INNER
GROUP BY INNER.ACCOUNT_ID ) AS OUTER ON ACC.ACCOUNT_ID = OUTER.ACCOUNT_ID
LEFT JOIN (SELECT ACCOUNT_ID, COUNT(*) AS TOTAL_NUMBER_OF_IN_KIND_GIFTS , SUM(OPPORTUNITY_AMOUNT) AS TOTAL_AMOUNT_OF_IN_KIND_GIFTS FROM REPORTING_DB.SEMANTIC.ALL_REVENUE_V
WHERE OPPORTUNITY_CAMPAIGN_DIVISIONAL_PROGRAM IN ('Type_Kind','Type_Gifts') OR OPPORTUNITY_DONATIONTYPE = 'Type_Gifts'
OR OPPORTUNITY_VEHICLE_TYPE IN ('Type_Kind','Type_Gifts') GROUP BY ACCOUNT_ID) AS ARV1 ON ACC.ACCOUNT_ID = ARV1.ACCOUNT_ID
WHERE ACC.ACCOUNT_ISDELETED = FALSE''', Kindly give the response strictly in this format only: OPTIMIZED CODE: ```optimized code``` and then, EXPLAINATI
ON: explaination on how you optimized it. The most import instruction is that output of original query and optimized query should be exactly same and stick to specified format."
"""

In [127]:
prompt = r"""Construct a DBT view model in the 'staging' schema, linking 'jaffle_shop' orders with payment information from 'stg_payments', summarizing successful payments per order, and merging with order details to handle potential missing payment scenarios."""

In [171]:
prompt = prompt.replace("'", "''")
prompt = prompt.replace("\n", " ")

In [167]:
prompt

"Generate documentation for following dbt code  {{ config(materialized=''table'') }}  with source as (     select * from {{ source(''src_table'', ''employees'') }} ),  parsed as (     select         id,         PARSE_JSON(json_column):field1::varchar as field1,         PARSE_JSON(json_column):field2::int as field2,         PARSE_JSON(json_column):field3::timestamp as field3     from source )  select * from parsed"

In [101]:
prompt = "Generate bdt code to join supply and demand column"

In [86]:
temp_arg ={"prompt":prompt, "max_tokens": 2000, "temperature": 0, "model": "/data/magicoder_model"}

In [87]:
temp_arg

{'prompt': "Construct a DBT view model in the 'staging' schema, linking 'jaffle_shop' orders with payment information from 'stg_payments', summarizing successful payments per order, and merging with order details to handle potential missing payment scenarios.",
 'max_tokens': 2000,
 'temperature': 0,
 'model': '/data/magicoder_model'}

In [113]:
type(arg)

str

In [172]:
arg = '{"prompt":'+'"'+prompt+'"'+', "max_tokens": 2000, "temperature": 0, "model": "/data/magicoder_model"}'
warehouse = 'USE WAREHOUSE TUTORIAL_WAREHOUSE;'
query = f"SELECT magicoder_udf(parse_json('{arg}'));"
print(query)

SELECT magicoder_udf(parse_json('{"prompt":""Assume you are a DBT query optimizer, Optimize this DBT query and if possible modularize this query using CTE: '''''' SELECT ACC.ACCOUNT_ID AS ACCOUNT_ID ,CASE WHEN OUTER.ACCOUNT_ID IS NOT NULL THEN ''Y'' ELSE ''N'' END AS DONATED ,NVL(NULLIF(SPLIT_PART(OUTER.FIRST_OPPORTUNITY_DTL, ''|'', 1), 0),0) AS FIRST_GIFT_AMOUNT ,NULLIF(SPLIT_PART(OUTER.FIRST_OPPORTUNITY_DTL, ''|'', 2),'''') AS FIRST_GIFT_DATE ,NVL(OUTER.TOTAL_AMOUNTS_OF_GIFTS_ALL_TIME_HC,0) AS TOTAL_AMOUNTS_OF_GIFTS_ALL_TIME_HC ,NVL(OUTER.TOTAL_NUMBER_OF_GIFTS_ALL_TIME_HC,0) AS TOTAL_NUMBER_OF_GIFTS_ALL_TIME_HC ,NVL(OUTER.TOTAL_AMOUNTS_OF_GIFTS_ALL_TIME_SC,0) AS TOTAL_AMOUNTS_OF_GIFTS_ALL_TIME_SC ,NVL(OUTER.TOTAL_NUMBER_OF_GIFTS_ALL_TIME_SC,0) AS TOTAL_NUMBER_OF_GIFTS_ALL_TIME_SC FROM ""DIM_DB"".""CORE"".""DIM_ACCOUNT"" ACC LEFT JOIN (SELECT INNER.ACCOUNT_ID ,ANY_VALUE(INNER.FIRST_OPPORTUNITY_DTL) AS FIRST_OPPORTUNITY_DTL ,ANY_VALUE(INNER.LAST_OPPORTUNITY_DTL) AS LAST_OPPORTUNITY_DTL

In [169]:
cur.execute(warehouse)
cur.execute(query)
result = cur.fetchall()
print(json.loads(result[0][0])['choices'][0]['text'])



```

This dbt code is using the `PARSE_JSON` function to parse a JSON column and extract specific fields. The `PARSE_JSON` function is a Snowflake specific function that converts a JSON string into a JSON object.

The `source` CTE is selecting from the `employees` table in the `src_table` schema.

The `parsed` CTE is selecting the `id` field from the `source` CTE, and then parsing the `json_column` field as a JSON object and extracting the `field1`, `field2`, and `field3` fields. The `::varchar`, `::int`, and `::timestamp` are type casts to convert the fields to the appropriate data types.

Finally, the main query is selecting all fields from the `parsed` CTE.

The documentation for this dbt code would be:

```
This dbt code is using the PARSE_JSON function to parse a JSON column and extract specific fields.

The source CTE is selecting from the employees table in the src_table schema.

The parsed CTE is selecting the id field from the source CTE, and then parsing the json_column fie

In [158]:
result

[('{\n  "choices": [\n    {\n      "finish_reason": "stop",\n      "index": 0,\n      "logprobs": null,\n      "text": ""\n    }\n  ],\n  "created": 900528,\n  "id": "cmpl-691f59d7581040858ef60c101927dfdd",\n  "model": "/data/magicoder_model",\n  "object": "text_completion",\n  "usage": {\n    "completion_tokens": 1,\n    "prompt_tokens": 74,\n    "total_tokens": 75\n  }\n}',)]

In [7]:
import requests
import json

# Obtain a session token.
token_data = conn._rest._token_request('ISSUE')
token_extract = token_data['data']['sessionToken']

# Create a request to the ingress endpoint with authz.
token = f'\"{token_extract}\"'
# Set this to the ingress endpoint URL for your service
url = 'http://gowiocnz-zqiseam-ltifosforscsaws.snowflakecomputing.app/vllm/magicoder/v1/completions'

In [8]:
headers = {
  'Content-Type': 'application/json', 'Authorization': f'Snowflake Token={token}'
}
print(headers)
print(url)
#print(payload)

{'Content-Type': 'application/json', 'Authorization': 'Snowflake Token="ver:1-hint:14544919633592326-ETMsDgAAAY5Vep50ABRBRVMvQ0JDL1BLQ1M1UGFkZGluZwEAABAAEMk0A8Nx2TZnVqLTbZSljBcAAACA48T2Lr8JKf/dNoQ+63nJH9+QxeHcASiPik2NSxujbZIIjM54+IsUlHcpGUPDwSwQ1JRZTrKIgY0Yuui3WXfMCELevmH/ouvrrg2R/v+F9ovWKUZG0EcY9IHo/Wvqgd1AEOopoYMmfVV+nCaf6UMf9zYA8pcF9Bp2QhsVDUBPBOEAFLlv+fQcuY44iMx5FEtWCNf9RUCD"'}
http://gowiocnz-zqiseam-ltifosforscsaws.snowflakecomputing.app/vllm/magicoder/v1/completions


In [9]:
payload = json.dumps({
  "prompt": "write a python code to check whether given string is pallindrome",
  "max_tokens": 2400,
  "temperature": 0,
  "model": "/data/magicoder_model"
})
headers = {
  'Content-Type': 'application/json', 'Authorization': f'Snowflake Token={token}'
}

response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)


{"detail":"Method Not Allowed"}
